In [1]:
!pip install searchtweets

In [2]:
!pip install textblob

In [3]:
!pip install afinn

In [4]:
import pandas as pd
import numpy as np
import requests
import searchtweets
from textblob import TextBlob
from afinn import Afinn
from datetime import *
import re

In [5]:
df = pd.read_csv("~/work/projetpy/fichiers_csv/tweets_2021-3-1_to_2021-4-1.csv")
#afinn = Afinn()
#afinn_scores = [afinn.score(text) for text in df.text]
#df['afinn'] = afinn_scores
#df[['afinn', 'text']].head()

In [10]:
b = TextBlob("Je suis en colère")
b.detect_language()
b.sentiment

Sentiment(polarity=0.0, subjectivity=0.0)

In [6]:
df.head()

,id,date,screenName,text,type
0,1377376779178967041,2021-03-31 21:46:29,ratonlave57,Pourquoi ne donne t’on pas de l’Aspegic pédiat...,tweet
1,1377339517170933768,2021-03-31 19:18:25,MathildeSlv,@mvrinesv Le syndrome grippal du Pfizer existe...,tweet
2,1377337776819990528,2021-03-31 19:11:30,EBrussat,@montagnethiers #Orleat 210 personnes #vaccine...,tweet
3,1377336854379331584,2021-03-31 19:07:50,garrec_pierre,"En tout cas vendredi, c’est le vaccin astrazen...",tweet
4,1377325612134699012,2021-03-31 18:23:10,FPasquisDumont,1700 centres Pfizer et Moderna...\nPlus de 70 ...,tweet


# Création des métadonnées

Nombre de hashtags

In [7]:
def decompte_hashtags(text):
    exp_match = re.compile("([#]\w+)")
    res = exp_match.findall(text)
    return len(res)

df["nbre_hashtags"] = df["text"].apply(lambda x : decompte_hashtags(x))

Nombre d'url

In [8]:
def decompte_url(text):
    exp_match = re.compile("([https//:]\w+[.]\w+)")
    res = exp_match.findall(text)
    return len(res)

df["nbre_url"] = df["text"].apply(lambda x : decompte_url(x))

Nombre de points d'exclamtion

In [10]:
def decompte_exclamation(text):
    exp_match = re.compile("(\w?\s?[!])")
    res = exp_match.findall(text)
    return len(res)

df["nbre_exclamation"] = df["text"].apply(lambda x : decompte_exclamation(x))


In [11]:
def decompte_suspension(text):
    exp_match = re.compile("(\w?\s?[.]{2,5})")
    res = exp_match.findall(text)
    return 0 if len(res)== 0 else 1
df["presence_suspension"] = df["text"].apply(lambda x : decompte_suspension(x))
df.head()

,id,date,screenName,text,type,nbre_hashtags,nbre_url,nbre_exclamation,presence_suspension
0,1377376779178967041,2021-03-31 21:46:29,ratonlave57,Pourquoi ne donne t’on pas de l’Aspegic pédiat...,tweet,0,0,0,0
1,1377339517170933768,2021-03-31 19:18:25,MathildeSlv,@mvrinesv Le syndrome grippal du Pfizer existe...,tweet,0,0,0,0
2,1377337776819990528,2021-03-31 19:11:30,EBrussat,@montagnethiers #Orleat 210 personnes #vaccine...,tweet,5,0,2,0
3,1377336854379331584,2021-03-31 19:07:50,garrec_pierre,"En tout cas vendredi, c’est le vaccin astrazen...",tweet,0,0,0,0
4,1377325612134699012,2021-03-31 18:23:10,FPasquisDumont,1700 centres Pfizer et Moderna...\nPlus de 70 ...,tweet,0,0,0,1


# Nettoyage des données

In [191]:
# fonction pour la transformation du texte (retrait des caractères spéciaux, des émojis, de la ponctuation, tranformation des majuscules en minuscule, etc)
def transforme(texte):
    texte = texte.lower() # mettre les mots en minuscule
    texte = re.sub(r"[,\!\?\%\(\)\/\"\$\£\-\+\*\µ,\:\&\§]", "", texte) # supprimer ",", "!", "?", "%", "(",")","/",'"', "$","£", "_", "-", "+", "*", "µ", ":","&,"§" 
    # retirer les hashtags #
    for item in re.compile("([#])").findall(texte):
        texte=texte.replace(item, "")
    # retirer les apostrophes '
    for item in re.compile("(['])").findall(texte):
        texte=texte.replace(item, " ")
    # retirer les points de suspension
    for item in re.compile("(\w?\s?[.]{1,5})").findall(texte):
        texte=texte.replace(item, "") 
    # retirer les personnes tagées
    for item in re.compile("([@]\w+)").findall(texte):
        texte=texte.replace(item, "")
    # retirer les adresses mail
    for item in re.findall('\S+@\S+', texte) :
        texte=texte.replace(item, "")
    # retire les mots contenant des chiffres
    for item in re.findall('\S+\d\S+', texte) :
        texte=texte.replace(item, "")
    #retirer les liens
    for item in re.compile("([https//:]\w+[.]\w+)").findall(texte):
        texte=texte.replace(item, "")
    return texte
    

In [192]:
df["texte1"]=df["text"].apply(lambda x: transforme(x))
df.head()

,id,date,screenName,text,type,nbre_hashtags,nbre_url,nbre_exclamation,presence_suspension,texte1
0,1377376779178967041,2021-03-31 21:46:29,ratonlave57,Pourquoi ne donne t’on pas de l’Aspegic pédiat...,tweet,0,0,0,0,pourquoi ne donne t’on pas de l’aspegic pédiat...
1,1377339517170933768,2021-03-31 19:18:25,MathildeSlv,@mvrinesv Le syndrome grippal du Pfizer existe...,tweet,0,0,0,0,le syndrome grippal du pfizer existe aussi au...
2,1377337776819990528,2021-03-31 19:11:30,EBrussat,@montagnethiers #Orleat 210 personnes #vaccine...,tweet,5,0,2,0,orleat personnes vaccinees moderna depuis lu...
3,1377336854379331584,2021-03-31 19:07:50,garrec_pierre,"En tout cas vendredi, c’est le vaccin astrazen...",tweet,0,0,0,0,en tout cas vendredi c’est le vaccin astrazene...
4,1377325612134699012,2021-03-31 18:23:10,FPasquisDumont,1700 centres Pfizer et Moderna...\nPlus de 70 ...,tweet,0,0,0,1,centres pfizer et modern\nplus de 70 an\nle 1...


In [195]:
help(re.UNICODE)

Help on RegexFlag in module re object:

class RegexFlag(enum.IntFlag)
 |  RegexFlag(value, names=None, *, module=None, qualname=None, type=None, start=1)
 |  
 |  An enumeration.
 |  
 |  Method resolution order:
 |      RegexFlag
 |      enum.IntFlag
 |      builtins.int
 |      enum.Flag
 |      enum.Enum
 |      builtins.object
 |  
 |  Data and other attributes defined here:
 |  
 |  ASCII = re.ASCII
 |  
 |  DEBUG = re.DEBUG
 |  
 |  DOTALL = re.DOTALL
 |  
 |  IGNORECASE = re.IGNORECASE
 |  
 |  LOCALE = re.LOCALE
 |  
 |  MULTILINE = re.MULTILINE
 |  
 |  TEMPLATE = re.TEMPLATE
 |  
 |  UNICODE = re.UNICODE
 |  
 |  VERBOSE = re.VERBOSE
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors inherited from enum.Enum:
 |  
 |  name
 |      The name of the Enum member.
 |  
 |  value
 |      The value of the Enum member.
 |  
 |  ----------------------------------------------------------------------
 |  Readonly properties inherited fro

In [196]:
# Tokenisation en considérant un token comme un mot
!pip install nltk
import nltk
nltk.download('puntk')
liste_mots= nltk.word_tokenize(df["texte1"], language='french')

[nltk_data] Error loading puntk: Package 'puntk' not found in index


LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/PY3/french.pickle[0m

  Searched in:
    - '/home/jovyan/nltk_data'
    - '/opt/conda/nltk_data'
    - '/opt/conda/share/nltk_data'
    - '/opt/conda/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************
